In [2]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, Input, Output, dash_table as dt
import plotly.express as px
import pandas as pd
import base64

# CRITICAL LINE - This configures JupyterDash for Codio
JupyterDash.infer_jupyter_proxy_config()

from animalshelter import AnimalShelter

# Data Manipulation / Model
username = "aacuser"
password = "AAC_ADMIN"
db = AnimalShelter(username, password, host="localhost", port=27017, db="AAC")

# Load initial data
df = pd.DataFrame.from_records(db.read({}))
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)
print("Number of records:", len(df))
print(df.head())

# Dashboard Layout / View
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

# Logo
image_filename = "Logo.png"
try:
    encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()
    logo_img = html.Img(src=f"data:image/png;base64,{encoded_image}", style={"height": "150px"})
except Exception as e:
    logo_img = html.Div(f"Logo missing or failed to load: {e}", style={"color": "red"})

app.layout = html.Div([
    html.Center(html.B(html.H1("SNHU CS-340 Dashboard"))),
    html.Center(html.B(html.H1("Project 2 Dashboard: Chris Clark"))),
    html.Hr(),
    logo_img,
    dcc.RadioItems(
        id="filter-type",
        options=[
            {"label": "Water Rescue", "value": "water"},
            {"label": "Mountain/Wilderness Rescue", "value": "mount"},
            {"label": "Disaster Rescue and Individual Tracking", "value": "disaster"},
            {"label": "Reset", "value": "reset"},
        ],
        value="reset",
        style={"textAlign": "center"}
    ),
    html.Hr(),
    dt.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action="native",
        sort_action="native",
        column_selectable="single",
        row_selectable="single",
        selected_columns=[],
        selected_rows=[],
        style_table={"height": "300px", "overflowY": "auto"},
    ),
    html.Div(className="row", style={"display": "flex"}, children=[
        html.Div(id="graph-id", className="col s12 m6"),
        html.Div(id="map-id", className="col s12 m6")
    ])
])

@app.callback(
    [Output("datatable-id", "data"),
     Output("datatable-id", "columns")],
    [Input("filter-type", "value")]
)
def update_dashboard(filter_type):
    try:
        query = {}
        if filter_type == "water":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
            }
        elif filter_type == "mount":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
            }
        elif filter_type == "disaster":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
            }

        dff = pd.DataFrame.from_records(db.read(query))
        if "_id" in dff.columns:
            dff.drop(columns=["_id"], inplace=True)
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in (dff.columns if not dff.empty else df.columns)]
        data = (dff if not dff.empty else df).to_dict('records')
        return data, columns
    except Exception as e:
        err = {"error": str(e)}
        return [err], [{"name": "error", "id": "error"}]

@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_virtual_data")
)
def update_graphs(viewData):
    try:
        dff = pd.DataFrame(viewData) if viewData else df
        if dff.empty or "breed" not in dff.columns:
            return html.P("No data to display or 'breed' column missing")
        fig = px.histogram(dff, x="breed", title="Available Animals by Breed")
        return [dcc.Graph(figure=fig)]
    except Exception as e:
        return [html.Pre(f"Graph error: {e}", style={"color": "red"})]

@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    try:
        dff = pd.DataFrame(viewData) if viewData else df
        if dff.empty or "location_lat" not in dff.columns or "location_long" not in dff.columns:
            return [dl.Map(style={"width": "1000px", "height": "500px"},
                           center=[30.75, -97.48], zoom=10,
                           children=[dl.TileLayer()])]
        row = selected_rows[0] if selected_rows else 0
        if row >= len(dff):
            row = 0
        lat = float(dff.iloc[row]["location_lat"])
        lon = float(dff.iloc[row]["location_long"])
        name = dff.iloc[row].get("name", "No Name")
        breed = dff.iloc[row].get("breed", "No Breed")

        return [dl.Map(style={"width": "1000px", "height": "500px"},
                       center=[lat, lon], zoom=10,
                       children=[
                           dl.TileLayer(),
                           dl.Marker(position=[lat, lon], children=[
                               dl.Tooltip(breed),
                               dl.Popup([html.H4("Animal Name"), html.P(name)])
                           ])
                       ])]
    except Exception as e:
        return [html.Pre(f"Map error: {e}", style={"color": "red"})]

# open dashboard
app.run_server(mode='jupyterlab', port=8054)

Connected to MongoDB
Number of records: 10000
     age_upon_outcome animal_id animal_type                    breed   
0  3          2 years   A716330         Dog  Chihuahua Shorthair Mix  \
1  5          2 years   A691584         Dog   Labrador Retriever Mix   
2  1          3 years   A746874         Cat   Domestic Shorthair Mix   
3  2           1 year   A725717         Cat   Domestic Shorthair Mix   
4  4         7 months   A733653         Cat              Siamese Mix   

          color date_of_birth             datetime            monthyear   
0   Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00  \
1     Tan/White    2012-11-06  2015-05-30 13:48:00  2015-05-30T13:48:00   
2   Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
3  Silver Tabby    2015-05-02  2016-05-06 10:49:00  2016-05-06T10:49:00   
4    Seal Point    2016-01-25  2016-08-27 18:11:00  2016-08-27T18:11:00   

    name outcome_subtype     outcome_type sex_upon_outcome  location_l